# Analyse des retards au départ de Paris-CDG – Été 2022  
**Projet Data Analyst – Simulation et étude opérationnelle**

Ce notebook présente un projet complet de data analyse basé sur une situation réelle :  
la crise opérationnelle de l’été 2022 à l’aéroport Paris-Charles de Gaulle (CDG), marquée par une reprise rapide du trafic et de fortes perturbations (retards, pertes de bagages, sous-effectifs).

### 🎯 Objectifs du projet
- **Simuler un dataset réaliste** de 500 vols au départ de CDG entre juin et août 2022.  
- **Analyser la ponctualité (OTP)** selon différents axes :  
  - plage horaire,  
  - terminal,  
  - type de vol,  
  - cause de retard,  
  - avant / après un incident bagages simulé début juillet 2022.  
- **Identifier les sources principales de perturbation** et les zones critiques dans les opérations sol.  
- **Proposer des actions concrètes** pour améliorer la fiabilité du hub et anticiper les pics de trafic.

### 🧰 Compétences mises en œuvre
- Simulation et préparation de données (Python, Pandas).  
- Nettoyage, transformation et enrichissement de données.  
- Analyse descriptive (OTP, distributions, segmentation).  
- Visualisation (Matplotlib).  
- Interprétation orientée métier (opérations aériennes).  

Ce projet constitue une base solide pour un portfolio professionnel dans le domaine **Data / Aviation / Opérations**.


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Options d'affichage pour mieux lire les DataFrame
pd.set_option("display.max_columns", 50)

# On part du dossier où se trouve le notebook (notebook/)
# puis on remonte d'un niveau vers la racine du projet
PROJECT_ROOT = Path().resolve().parents[1]

# Dossier data à la racine du projet
DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

print("PROJECT_ROOT :", PROJECT_ROOT)
print("DATA_DIR     :", DATA_DIR)


PROJECT_ROOT : /Users/godwincherubin/Documents/GitHub/cdg-summer-2022-delay-analysis
DATA_DIR     : /Users/godwincherubin/Documents/GitHub/cdg-summer-2022-delay-analysis/data


## 1. Construction pédagogique du jeu de données

Dans un contexte réel, les données proviendraient du data warehouse d’une compagnie aérienne  
(ex : systèmes opérationnels, données AODB, logs bagages, fichiers équipages, etc.).

Dans ce projet, je reconstruis un jeu de données **pédagogique mais réaliste**, inspiré de la crise opérationnelle de l’été 2022 à Paris-CDG :

- **vols_cdg_ete2022** : liste de vols au départ de CDG entre juin et août 2022 (500 vols simulés),
- **causes_retard** : cause principale de retard pour chaque vol (bagages, handling, ATC, météo…).

Ces données permettent de reproduire un raisonnement professionnel d’analyste opérations :
- impact d’un incident bagages,
- évolution de la ponctualité (OTP),
- analyse par terminal, par plage horaire, par type de vol,
- identification des zones critiques dans le hub.

La première étape consiste à générer les **dates des vols**, base du dataset.


In [13]:
# Nombre total de vols à simuler
n_flights = 500

# Générateur aléatoire (pour avoir toujours les mêmes résultats)
rng = np.random.default_rng(42)

# Période de l'étude : 1er juin → 31 août 2022
date_range = pd.date_range("2022-06-01", "2022-08-31", freq="D")

# On tire aléatoirement 500 dates dans cette période
dates_vol = rng.choice(date_range, size=n_flights)

# Création d'un DataFrame de base
df = pd.DataFrame({
    "date_vol": pd.to_datetime(dates_vol).strftime("%Y-%m-%d")
})

df.head()


,date_vol
0,2022-06-09
1,2022-08-11
2,2022-07-31
3,2022-07-11
4,2022-07-10
